In [1]:
import tkinter as tk
import numpy as np
from tkinter import Scale,ttk,Button,Text
from PIL import Image, ImageTk
import os

In [11]:
imageList=[]
folder=os.getcwd()+"\\images"
for filename in os.listdir(folder):
    img=np.asarray(Image.open(os.path.join(folder,filename)))
    if len(img.shape)<3: # Gif or Greyscale
        continue
    if img.shape[2]>3:
        img=img[:,:,:3]
    if img is not None:
        imageList.append(img)
print(len(imageList))
if not os.path.exists(os.getcwd()+"\\heads"):
    os.mkdir(os.getcwd()+"\\heads")

321


In [12]:
def square_update(length):
    global currentIndex,canvas,source,photoImg,origin
    
    length=int(length)
    
    if origin[0]+length>=len(source):
        origin[0]=len(source)-1-length
    if origin[1]+length>=len(source[0]):
        origin[1]=len(source[1])-1-length    

    toDraw=source.copy()
    red=[255,0,0]
        
    for i in range(origin[0],origin[0]+length):
        if i>=len(source):
            i=len(source)-1
        toDraw[i][origin[1]]=red
        x=origin[1]+length
        if x>=len(source[0]):
            x=len(source[0])-1
        toDraw[i][x]=red
    for j in range(origin[1],origin[1]+length):
        if j>=len(source[0]):
            j=len(source[0])-1
        toDraw[origin[0]][j]=red
        x=origin[0]+length
        if x>=len(source):
            x=len(source)-1
        toDraw[x][j]=red
        
    image=Image.fromarray(toDraw)
    image=image.resize((canvas_width,canvas_height),resample=0)
    photoImg=ImageTk.PhotoImage(image,master=canvas)
    canvas.create_image(0,0,image=photoImg,anchor="nw")

In [13]:
def cropCurrent():
    global source,origin,file_count,lengthSlider,text
    length=lengthSlider.get()
    sub_image=source[origin[0]:origin[0]+length,origin[1]:origin[1]+length]
    image=Image.fromarray(sub_image)
    image.save("heads\\"+str(file_count)+"_head.jpg")
    print("Cropped image saved at","heads\\"+str(file_count)+"_head.jpg")
    file_count+=1
    text.delete('1.0',tk.END)
    text.insert(tk.END,"Image Number: "+str(currentIndex)+"\nFiles Saved: "+str(file_count))

In [14]:
def Up(event):
    global pixel_per_move
    pixel_per_move=int(pixel_per_move)
    if origin[0]-pixel_per_move>=0:
        origin[0]=origin[0]-pixel_per_move
    else:
        origin[0]=0
    square_update(lengthSlider.get())
def Down(event):
    global pixel_per_move
    pixel_per_move=int(pixel_per_move)
    if origin[0]+pixel_per_move<len(source)-1:
        origin[0]=origin[0]+pixel_per_move
    else:
        origin[0]=len(source)-1
    square_update(lengthSlider.get())
def Left(event):
    global pixel_per_move
    pixel_per_move=int(pixel_per_move)
    if origin[1]-pixel_per_move>0:
        origin[1]=origin[1]-pixel_per_move
    else:
        origin[1]=0
    square_update(lengthSlider.get())
def Right(event):
    global pixel_per_move
    pixel_per_move=int(pixel_per_move)
    if origin[1]+pixel_per_move<len(source[0])-1:
        origin[1]=origin[1]+pixel_per_move
    else:
        origin[1]=len(source[0])-1
    square_update(lengthSlider.get())

In [15]:
def loadImg():
    global currentIndex,canvas,lengthSlider,source,photoImg,canvas_height,origin,text,file_count
    if currentIndex==len(imageList):
        print("Last image, program ends here.")
        root.destroy()
        return
    canvas.delete('all')
    source=imageList[currentIndex]
    currentIndex+=1
    text.delete('1.0',tk.END)
    text.insert(tk.END,"Image Number: "+str(currentIndex)+"\nFiles Saved: "+str(file_count))
    image=Image.fromarray(source)
    size=image.size
    max_length=min(size)-1
    length=max_length
    canvas_height=int(size[1]*(canvas_width/size[0]))
    origin=[0,0]
    image=image.resize((canvas_width,canvas_height),resample=0)
    canvas.config(height=canvas_height)
    photoImg=ImageTk.PhotoImage(image,master=canvas)
    canvas.create_image(0,0,image=photoImg,anchor="nw")
    lengthSlider.config(to=max_length)
    lengthSlider.set(max_length/2)

In [16]:
def loadPrevious():
    global currentIndex,text
    if currentIndex>1:
        currentIndex-=2
        loadImg()
    else:
        text.insert(tk.END,"\nNo Previous image!")

In [17]:
def movement_update(value):
    global pixel_per_move
    pixel_per_move=value

In [232]:
currentIndex=0
file_count=0
origin=[0,0]
length=None
canvas_width=640
pixel_per_move=1
canvas_height=None

root=tk.Tk()

canvas=tk.Canvas(root,width=canvas_width,height=canvas_width,borderwidth=0,highlightthickness=0)
canvas.pack()

crop=Button(root,text="Crop",command=cropCurrent)
crop.pack(side="bottom")

moveOn=Button(root,text="Next",command=loadImg)
moveOn.pack(side="bottom")

moveBack=Button(root,text="Previous",command=loadPrevious)
moveBack.pack(side="bottom")
movementSlider=None
movementSlider=Scale(root,variable=pixel_per_move,orient="vertical",from_=1,to=5, command=lambda value=movementSlider:movement_update(value))
movementSlider.set(1)
movementSlider.pack(side="right")

def scroll_movement(event):
    movementSlider.set(movementSlider.get()+1*(event.delta/120))
root.bind("<MouseWheel>", scroll_movement)

lengthSlider=None
lengthSlider=Scale(root,variable=length,orient="horizontal",from_=1,command=lambda value=lengthSlider:square_update(value),length=canvas_width*0.6)
lengthSlider.set(int(lengthSlider.cget("to")/2))
lengthSlider.pack(side="bottom")

text=Text(root,height=3)
text.insert(tk.END,"Image Number: 0\nFiles Saved:0")
text.pack()

root.bind("<Up>", Up)
root.bind("<Down>", Down)
root.bind("<Left>", Left)
root.bind("<Right>", Right)
root.bind("w", Up)
root.bind("s", Down)
root.bind("a", Left)
root.bind("d", Right)

loadImg()
square_update(lengthSlider.get())
root.after(1, lambda: root.focus_force())
root.mainloop()

Cropped image saved at heads\0_head.jpg
Cropped image saved at heads\1_head.jpg
Cropped image saved at heads\2_head.jpg
Cropped image saved at heads\3_head.jpg
Cropped image saved at heads\4_head.jpg
Cropped image saved at heads\5_head.jpg
Cropped image saved at heads\6_head.jpg
Last image, program ends here.
